## 加载模块

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask
from dask_ml.preprocessing import DummyEncoder
from dask_ml.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from dask_glm.datasets import make_classification
import gc

In [3]:
gc.collect()

0

## 加载数据

### 其它

In [4]:
path3 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\test_format1.csv'

In [5]:
df_test1 = dd.read_csv(path3)
df_test1.sample(5) 

,user_id,merchant_id,prob
npartitions=1,,,
,int64,int64,float64
,...,...,...


### train1数据

In [6]:
path4 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\train_format1.csv'

In [7]:
df_train1 = dd.read_csv(path4)

In [25]:
df_train1.head(5)

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [27]:
df_train1[df_train1['label'].isnull()].compute()
# 上述生成的是空集

,user_id,merchant_id,label


### user_info数据

In [7]:
path5 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1.csv'

In [8]:
user_info = dd.read_csv(path5)

In [30]:
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


### 1.78G的user_log数据导入

In [8]:
path6 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_log_format1.csv'   

In [9]:
user_log_data = dd.read_csv(path6)

In [33]:
user_log_data.head(6)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0
5,328862,623866,1271,2882,2661.0,829,0


## 处理user_log数据

### user_id被记录的信息条数 

In [10]:
u1 = user_log_data[['user_id']].copy()
u1 = u1.groupby('user_id').size().reset_index()
u1 = u1.rename(columns = {0:'user_action_count'})

In [35]:
u1.head()

,user_id,user_action_count
0,16,67
1,19,114
2,56,281
3,155,66
4,201,53


### 根据user_id对action_type编码

In [11]:
u3 = user_log_data[['user_id','action_type']].copy()

# set(u3.action_type.isnull()).compute()
# 结果为False，说明actiio_type中没有空值

u3 = u3.categorize(columns = ['action_type'])

u3 = DummyEncoder().fit_transform(u3)

In [39]:
u3.head()

,user_id,action_type_0,action_type_2,action_type_3,action_type_1
0,328862,1,0,0,0
1,328862,1,0,0,0
2,328862,1,0,0,0
3,328862,1,0,0,0
4,328862,1,0,0,0


### 对user_id操作的天数计数

In [12]:
u4 = user_log_data[['user_id','time_stamp']].copy()

u4 = u4[['user_id','time_stamp']].drop_duplicates()

u4 = u4.groupby(['user_id']).size().reset_index()

u4 = u4.rename(columns = {0:'user_handle_day_count'})

In [41]:
u4_h = u4.loc[:5,:].copy()
u4_h.compute() # 为什么这一步的时候那么的消耗内存呢？

,user_id,user_handle_day_count
0,1,5
1,2,9
2,3,13
3,4,10
4,5,30
5,6,56


### 每个user_id对每个item_id(具体的商品)的操作数量编码

In [13]:
u5 = user_log_data[['user_id','item_id','action_type']].copy()

u5 = u5.categorize(columns = ['action_type'])

u5 = DummyEncoder().fit_transform(u5)

u5 = u5.groupby(['user_id','item_id']).sum().reset_index()

In [46]:
u5_h = u5.loc[:4,:].copy() # 只是这里生成的行索引没有重排，不知道改怎么办！
u5_h.compute()

,user_id,item_id,action_type_0,action_type_2,action_type_3,action_type_1
0,16,51942,1.0,0.0,0.0,0.0
1,16,64543,6.0,1.0,0.0,0.0
2,16,83910,1.0,0.0,0.0,0.0
3,16,140170,1.0,0.0,0.0,0.0
4,16,159165,1.0,0.0,0.0,0.0


### 每个user_id对每个brand_id的操作编码

In [14]:
u6 = user_log_data[['user_id','brand_id','action_type']].copy()

u6 = u6.categorize(columns = ['action_type'])

u6 = DummyEncoder().fit_transform(u6)
u6 = u6.groupby(['user_id','brand_id']).sum().reset_index()

In [50]:
u6.loc[:3,:].compute()

,user_id,brand_id,action_type_0,action_type_2,action_type_3,action_type_1
0,16,277.0,1.0,0.0,0.0,0.0
1,16,625.0,1.0,0.0,0.0,0.0
2,16,654.0,2.0,0.0,0.0,0.0
3,16,672.0,4.0,0.0,0.0,0.0


### 每个user_id对每个seller_id下的item_id的action_type编码

In [15]:
u7 = user_log_data[['user_id','seller_id','item_id','action_type']].copy()
u7 = u7.rename(columns = {'action_type':'usi_action_type'})
u7 = u7.categorize(columns = ['usi_action_type'])
u7 = DummyEncoder().fit_transform(u7)
u7 = u7.groupby(['user_id','seller_id','item_id']).sum().reset_index()

In [55]:
u7.head()

,user_id,seller_id,item_id,usi_action_type_0,usi_action_type_2,usi_action_type_3,usi_action_type_1
0,16,574,197311,1.0,0.0,0.0,0.0
1,16,650,218217,1.0,0.0,0.0,0.0
2,16,650,293373,1.0,0.0,0.0,0.0
3,16,650,391126,1.0,0.0,0.0,0.0
4,16,650,673172,4.0,0.0,0.0,0.0


### 按照cat_id分类统计item_id的数量

In [16]:
i1 = user_log_data[['cat_id','item_id']].copy()
i1 = i1.drop_duplicates()
i1 = i1.groupby('cat_id').size().reset_index()

i1 = i1.rename( columns={0: 'cat_item_count'} ) # dask不支持参数inplace

In [20]:
i1_h = i1.loc[:5,:].copy() # 如果直接用i1.head()，那么笔记本8G内存就不够了。
i1_h.compute()

,cat_id,cat_item_count
0,1,34
1,2,2303
2,3,12
3,4,552
4,5,551
5,6,5


### 按照seller_id分类统计brand_id的数量

In [17]:
i2 = user_log_data[['seller_id','brand_id']].copy()
i2 = i2.drop_duplicates()
i2 = i2.groupby('seller_id').size().reset_index()
i2 = i2.rename( columns={0: 'seller_brand_count'} )

In [22]:
i2_h = i2.loc[:5,:].copy()
i2_h.compute()

,seller_id,seller_brand_count
0,1,3
1,2,2
2,3,2
3,4,3
4,5,2
5,6,3


### 按照seller_id统计item_id的个数

In [18]:
i3 = user_log_data[['seller_id','item_id']].copy()
i3 = i3.drop_duplicates()
i3 = i3.groupby('seller_id').size().reset_index()
i3 = i3.rename(columns = {0:'seller_item_id_count'})

In [62]:
i3.head()

,seller_id,seller_item_id_count
0,1,2977
1,2,154
2,3,171
3,4,155
4,5,660


### seller_id下cat_id的个数

In [19]:
i4 = user_log_data[['seller_id','cat_id']].copy()
i4 = i4.drop_duplicates()
i4 = i4.groupby('seller_id').size().reset_index()
i4 = i4.rename(columns = {0:'seller_cat_count'})

In [64]:
i4.head()

,seller_id,seller_cat_count
0,1,44
1,2,10
2,3,4
3,4,7
4,5,23


### cat_id下brand_id计数

In [20]:
i5 = user_log_data[['brand_id','cat_id']].copy()
i5 = i5.drop_duplicates()
i5 = i5.groupby('cat_id').size().reset_index()
i5 = i5.rename(columns = {0:'cat_brand_count'})

In [121]:
i5[i5.cat_id.isnull()].compute()

,cat_id,cat_brand_count


## 处理user_info数据

### 按照user_id分类对gender编码、对user_age_range编码

In [24]:
userinfo = dd.read_csv( 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1_handled.csv' ) 

In [29]:
u2 = userinfo.drop_duplicates()
u2 = u2.categorize( columns = ['age_range', 'gender'] )
u2 = DummyEncoder().fit_transform( u2 )

In [30]:
u2.head()

,user_id,age_range_6.0,age_range_5.0,age_range_4.0,age_range_7.0,age_range_3.0,age_range_0.0,age_range_2.0,age_range_1.0,gender_1.0,gender_0.0,gender_2.0
0,376517,1,0,0,0,0,0,0,0,1,0,0
1,234512,0,1,0,0,0,0,0,0,0,1,0
2,344532,0,1,0,0,0,0,0,0,0,1,0
3,186135,0,1,0,0,0,0,0,0,0,1,0
4,30230,0,1,0,0,0,0,0,0,0,1,0


## user_id、seller_id数据合并

In [31]:
# 将u1和ui1合并，u1：log数据按照user_id分类后计数，
# ui1：info数据按照user_id对age_range编码
u1u2 = u1.merge(u2,on = 'user_id') # 默认的how = inner

# 再将上面的u1ui1 和u3合并，u3：根据user_id对action_type编码
u1u2u3 = u1u2.merge(u3,on = 'user_id')

# 再将上面和u4合并，u4：将user_id分类，对其操作的天数计数
u1u2u3u4 = u1u2u3.merge(u4,on = 'user_id')

# 将上面的结果和u7合并，u7：根据user_id对每个seller_id下的item_id的action_type编码、计数
u1u2u3u4u7 = u1u2u3u4.merge(u7,on = 'user_id')

# 将上面的结果和i3合并，i3：根据seller_id对其下的item_id计数
u1u2u3u4u7i3 = u1u2u3u4u7.merge(i3,on = 'seller_id')

# 将上面的结果和i4合并，i4：根据seller_id对其下的cat_id计数
u1u2u3u4u7i3i4 = u1u2u3u4u7i3.merge(i4,on = 'seller_id') 

In [38]:
u1u2u3u4u7i3i4.to_csv('E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\u1ui1u3u2u4u7i3i4.csv',
                         index = False, encoding='utf-8' )

MemoryError: 

In [1]:
u1u2u3u4u7i3i4

NameError: name 'u1u2u3u4u7i3i4' is not defined

## train1集数据和user_seller数据合并 

In [43]:
usi = u1ui1u3u2u4u7i3i4.rename(columns={'seller_id':'merchant_id'})
# 因为在train1数据中的是merchant_id

In [126]:
usitrain1 = usi.merge(df_train1,on = ['user_id','merchant_id']) 

## 训练集模型测试

In [127]:
features = usitrain1.columns.difference(['user_id','merchant_id','label'])

In [49]:
usitrain1.columns

Index(['user_id', 'user_action_count', 'age_range_6.0', 'age_range_5.0',
       'age_range_4.0', 'age_range_7.0', 'age_range_3.0', 'age_range_0.0',
       'age_range_2.0', 'age_range_1.0', 'action_type_0', 'action_type_2',
       'action_type_3', 'action_type_1', 'gender_1.0', 'gender_0.0',
       'gender_2.0', 'user_handle_day_count', 'merchant_id', 'item_id',
       'usi_action_type_0', 'usi_action_type_2', 'usi_action_type_3',
       'usi_action_type_1', 'seller_item_id_count', 'seller_cat_count',
       'label'],
      dtype='object')

In [128]:
x = usitrain1[features.tolist()].fillna(value=0, method=None, limit=None, axis=0)
x = x.values

In [131]:
x

dask.array<values, shape=(nan, 24), dtype=int64, chunksize=(nan, 24)>

In [129]:
x.reshape((-1, 24))

ValueError: cannot convert float NaN to integer

In [110]:
type(x)

dask.array.core.Array

In [130]:
y = usitrain1['label'].fillna(value=0).values

In [132]:
y.reshape((-1, 1))

ValueError: cannot convert float NaN to integer

In [112]:
type(y)

dask.array.core.Array

In [115]:
lgr = LogisticRegression()

In [116]:
lgr_model = lgr.fit(x, y) 

NotImplementedError: Can not add intercept to array with unknown chunk shape

In [105]:
x, y = make_classification()

y

dask.array<lt, shape=(1000,), dtype=bool, chunksize=(100,)>

In [114]:
y_pre = lgr_model.predict(x)
accuracy_score(y,y_pre)

NameError: name 'lgr_model' is not defined

## 分类test中的数据

In [ ]:
usitest1 = usi.merge(df_test1,on = ['user_id','merchant_id'],how = 'right')

In [ ]:
x_test = usitest1[features]

In [ ]:
y_pre_test = lgr_model.predict(x_test)
prediction = df_test1.copy()
prediction['prob'] = y_pre_test

In [ ]:
prediciton.to_csv('prediction.csv',header=False,index=False)

至此即可将数据提交到比赛了。